# libs

In [2]:
import praw
import pandas as pd
import tqdm

# initialize

In [2]:
client_id = 'gkO-rNH3V7jnyAiWM8H_Dw'
client_secret = 'h6LPGT7SAsfo-RVckDY_yIyLTBbv8Q'
user_agent = 'beksultan_web_scrapper'

reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

In [4]:
print(reddit)

# my subreddits

In [7]:
datasets_path = './first-statistics/DP Datsets - Классы и датасеты.csv'
df = pd.read_csv(datasets_path)

my_subreddits = (
    df
    [df['Название датасета'] == 'Reddit']
    ['Ссылка на датасет']
    .str.split('/')
    .apply(lambda x: x[-2])
    .tolist()
)

#### new subreddits

In [ ]:
# first subbredits
# 'artificial', 'ArtificialInteligence',
# 'learnmachinelearning', 'MachineLearning',
# 'reinforcementlearning', 'computervision', 
# 'LanguageTechnology', 'celebrities', 'Celebs',
# 'CelebBattles', 'CelebrityCandids', 'depression',
# 'CatastrophicFailure', 'naturesfury', 'ThingsThatBlowUp', 
# 'DestructionPorn', 'StormComing', 'Conflagration',
# 'promptcritical', 'finance', 'economy', 'Crypto_Currency_News',
# 'jobs', 'careeradvice', 'work', 'GetEmployed', 'humanresources',
# 'cscareerquestions', 'Unemployed', 'UKJobs', 'psychology', 'askpsychology',
# 'AcademicPsychology', 'BehaviorAnalysis', 'religion', 'AskAChristian', 
# 'Christianity', 'TrueChristian', 'Christians', 'Bible', 'OrthodoxChristianity',
# 'Catholicism', 'atheism',
# 'AcademicBiblical', 'space', 'Astronomy', 'Mars', 'astrophotography',
# 'nasa', 'askastronomy', 'spacex'

# new subreddits 
# https://www.reddit.com/r/entertainment/
# https://www.reddit.com/r/CrazyDaysandNights/

# https://www.reddit.com/r/news/
# https://www.reddit.com/r/politics/
# https://www.reddit.com/r/relationship_advice/

# https://www.reddit.com/r/science/
# https://www.reddit.com/r/technology/

# https://www.reddit.com/r/Oscars/

# my_subreddits = ['science','technology']

# https://www.reddit.com/r/gadgets/

# https://www.reddit.com/r/leagueoflegends/
#     https://www.reddit.com/r/videogames/
#         https://www.reddit.com/r/DotA2/
#             https://www.reddit.com/r/GlobalOffensive/

# parser function

In [6]:
def subbredit_parser(subreddit):
    limit=5000
    except_type = praw.models.reddit.more.MoreComments
    posts = list()
    ml_subreddit = reddit.subreddit(subreddit)
    for post in ml_subreddit.hot(limit=limit):
        if post.num_comments > 0:
             comments = [comment.body for comment in post.comments.list() if type(comment)!=except_type]
        else:
            comments = None
        posts.append(
            [post.title, #post.score,
             #post.id,
             post.subreddit,
             #post.url, 
             post.num_comments,
             post.selftext, #post.created,
             comments
            ]
        )
    posts = pd.DataFrame(posts,
                         columns=['title', #'score', 'id',
                                  'subreddit', #'url',
                                  'num_comments',
                                  'body', #'created',
                                  'comments'])
    return posts

# main cell

In [7]:
result = pd.DataFrame()
for subrdt in tqdm.tqdm_notebook(my_subreddits):
    result = pd.concat([result, subbredit_parser(subreddit=subrdt)])

/tmp/ipykernel_134405/1079143492.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for subrdt in tqdm.tqdm_notebook(my_subreddits):


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
part1 = result[['title','subreddit']]
part2 = result[['body','subreddit']]
part3 = result[['comments','subreddit']]

part1 = part1.drop_duplicates('title')

part2 = part2[part2.body!=''].drop_duplicates('body')

part3 = part3.explode('comments').drop_duplicates('comments')

part1.columns = ['text','origin_topic']
part1['topic'] = part1.origin_topic.map({'science':'Science&Technology',
                                        'technology':'Science&Technology'})
part1['source'] = 'reddit-title-science-tech'

part2.columns = ['text','origin_topic']
part2['topic'] = part2.origin_topic.map({'science':'Science&Technology',
                                        'technology':'Science&Technology'})
part2['source'] = 'reddit-body-science-tech'

part3.columns = ['text','origin_topic']
part3['topic'] = part3.origin_topic.map({'science':'Science&Technology',
                                        'technology':'Science&Technology'})
part3['source'] = 'reddit-comments-science-tech'

res = pd.concat([part1,part2,part3])

res = res[['text', 'topic', 'origin_topic', 'source']].drop_duplicates('text')

res['topic'] = 'Science&Technology'

res.to_csv('./dataset-prepared-data/reddit-science-tech.csv', index=False)